In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import mysql.connector
import pandas as pd

In [2]:
# Common function defenition
from pandas import DataFrame
def connect_db():
    db_connection = mysql.connector.connect(
    host='146.56.43.43',
    user='ffbemaster',
    password='aksaksgo1!',
    database='ffbe')
    return db_connection
def fetch_data_from_db(sql):
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute(sql)
    res = cursor.fetchall()
    cursor.close()
    conn.close()
    return res
def fetch_data_from_db_with_columns_names(sql):
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute(sql)
    column_names = [column[0] for column in cursor.description]
    res = cursor.fetchall()
    cursor.close()
    conn.close()
    return column_names, res
def write_to_server(sql, values):
    conn = connect_db()
    cursor = conn.cursor()
    if type(values) != list:
        cursor.execute(sql, values)
    else:
        for i in range(len(values)):
            cursor.execute(sql, values[i])
    res = cursor.fetchall()
    cursor.close()
    conn.commit()
    conn.close()
    return res
def from_df_to_sql(df:DataFrame):
    conn = connect_db()
    col_names = df.columns
    pass

In [ ]:
def get_vc_for_job_to_excel():
    table_name = 'vc_for_job_list'
    excel_name_to_write = 'vc_for_job_from_server.xlsx'
    
    # Retrieve data from server, then save it to excel file. 
    sql_query = f"select * from {table_name}"
    column_names, data = fetch_data_from_db_with_columns_names(sql_query)
    data = fetch_data_from_db(sql_query)
    df = pd.DataFrame(data, columns=column_names, index=None)
    df.to_excel(excel_name_to_write, index=False)
    
def upload_vc_for_job_from_excel():
    # Retrieve data from excel file, then write it to server.
    table_name = 'vc_for_job_list'
    excel_name = 'vc_for_job_to_upload.xlsx'
    
    df = pd.read_excel(excel_name)
    df = df.fillna("")

    conn = connect_db()
    cursor = conn.cursor()

    col_names = df.columns.tolist()

    update_sql = "=%s, ".join(col_names)+"=%s"
    sql_query = f"""INSERT INTO {table_name} ({','.join(col_names)})
    values ({','.join(["%s"]*len(col_names))})
    ON DUPLICATE KEY UPDATE
    """ + update_sql
    # print(sql_query)
    for i,r in df.iterrows():
        # print(r.values)
        values = tuple(r.values)*2
        cursor.execute(sql_query, values)
    conn.commit()
    cursor.close()
    conn.close()

In [15]:
# Retrieve data from server, then save it to excel file. 
sql_query = "select * from vc_for_job_list"
column_names, data = fetch_data_from_db_with_columns_names(sql_query)
data = fetch_data_from_db(sql_query)
df = pd.DataFrame(data, columns=column_names, index=None)
df.to_excel("vc_for_job_from_server.xlsx", index=False)

In [3]:
# Retrieve data from excel file, then write it to server.
table_name = 'vc_for_job_list'

df = pd.read_excel("vc_for_job.xlsx")
df = df.fillna("")

conn = connect_db()
cursor = conn.cursor()

col_names = df.columns.tolist()

update_sql = "=%s, ".join(col_names)+"=%s"
sql_query = f"""INSERT INTO {table_name} ({','.join(col_names)})
values ({','.join(["%s"]*len(col_names))})
ON DUPLICATE KEY UPDATE
""" + update_sql
# print(sql_query)
for i,r in df.iterrows():
    # print(r.values)
    values = tuple(r.values)*2
    cursor.execute(sql_query, values)
conn.commit()
cursor.close()
conn.close()